In [1]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 7.8MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=6986d45007daec1e4ee478985c7d8a878075e9b940386b19aab7860e1b41ed5b
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=01c40cd35e3490fce0738278bfe4c9af993605fe76c16482b7fede6017a2b93d
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import pandas as pd
import numpy as np
df = pd.read_csv('/content/IMDB Dataset.csv',names=['sentence','label'])
df.head()

,sentence,label
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [0]:
df.loc[df['label'] == 'positive' , 'label'] = 1
df.loc[df['label'] == 'negative' , 'label'] = 0

In [0]:
from sklearn.model_selection import train_test_split
X = df.sentence.values
y = df.label.values
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [36]:
X.shape

(10000,)

In [6]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
vocab_size = len(tokenizer.word_index) + 1
print(X_train[2])
print(X_train_seq[2])

Using TensorFlow backend.


To experience Head you really need to understand where the Monkees were when they filmed it.<br /><br />This was as their series was coming to a close and the group was near break up. Their inventive and comedic series (sort of an American Idol of their day) took four unknown actors and formed a manufactured supergroup around them.<br /><br />This is their take on their "manufactured image" and status as the 2nd tier Beatles. They always felt they were in a box, trapped, and unable to find credibility despite their talents.<br /><br />It is also a hell of a musical-trippy, inventive (I have the soundtrack) and full of surprises.<br /><br />See it with an open mind.
[5, 577, 391, 21, 65, 351, 5, 403, 117, 1, 7688, 68, 50, 33, 755, 9, 7, 7, 11, 13, 14, 62, 222, 13, 545, 5, 3, 502, 2, 1, 555, 13, 854, 1034, 53, 62, 4377, 2, 1736, 222, 436, 4, 32, 271, 6743, 4, 62, 246, 541, 680, 1537, 151, 2, 5305, 3, 11138, 31119, 183, 91, 7, 7, 11, 6, 62, 193, 20, 62, 11138, 1345, 2, 2739, 14, 1, 3498, 

In [7]:
max_length = max([len(s.split()) for s in X])
print(max_length)

1830


In [8]:
from keras.preprocessing.sequence import pad_sequences
X_train_pad = pad_sequences(X_train_seq, padding='post', maxlen=max_length)
X_test_pad = pad_sequences(X_test_seq, padding='post', maxlen=max_length)
X_train_pad[0,:]

array([ 10, 192, 129, ...,   0,   0,   0], dtype=int32)

In [0]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath,encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [0]:
embedding_dim3 = 100
embedding_matrix = create_embedding_matrix('/content/glove.6B.100d.txt', tokenizer.word_index, embedding_dim3)

In [17]:
from keras.layers import Dropout
from keras.layers import Dense,Conv1D,Flatten,MaxPooling1D
from keras.models import Sequential
from keras import layers
from keras.layers import GlobalMaxPool1D

embedding_dim3 = 100

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim3, 
                           weights=[embedding_matrix], 
                           input_length=max_length, 
                           trainable=True))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1830, 100)         5448700   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1826, 128)         64128     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 5,514,129
Trainable params: 5,514,129
Non-trainable params: 0
_________________________________________________________________


In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
epochs = 5
embedding_dim = 100
maxlen = max_length
output_file = 'output.txt'

In [0]:
def create_model(num_filters, kernel_size, vocab_size, embedding_dim, maxlen):
    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen,weights=[embedding_matrix]))
    model.add(layers.Conv1D(num_filters, kernel_size, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [27]:
param_grid = dict(num_filters=[32, 64, 128],
                      kernel_size=[3, 5, 7],
                      vocab_size=[vocab_size],
                      embedding_dim=[embedding_dim],
                      maxlen=[maxlen])
model = KerasClassifier(build_fn=create_model,
                            epochs=epochs, batch_size=32)
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                              cv=4, verbose=1, n_iter=5)
grid_result = grid.fit(X_train_pad, y_train)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
6000/6000 [==============================] - 3s 524us/step - loss: 0.6720 - accuracy: 0.6262
Epoch 2/5
6000/6000 [==============================] - 3s 496us/step - loss: 0.4149 - accuracy: 0.8462
Epoch 3/5
6000/6000 [==============================] - 3s 488us/step - loss: 0.2353 - accuracy: 0.9213
Epoch 4/5
6000/6000 [==============================] - 3s 490us/step - loss: 0.1024 - accuracy: 0.9798
Epoch 5/5
2000/2000 [==============================] - 0s 105us/step
Epoch 1/5
6000/6000 [==============================] - 3s 508us/step - loss: 0.6568 - accuracy: 0.6305
Epoch 2/5
6000/6000 [==============================] - 3s 495us/step - loss: 0.3951 - accuracy: 0.8375
Epoch 3/5
6000/6000 [==============================] - 3s 496us/step - loss: 0.2308 - accuracy: 0.9117
Epoch 4/5
6000/6000 [==============================] - 3s 493us/step - loss: 0.1101 - accuracy: 0.9685
Epoch 5/5
2000/2000 [==============================] - 0s 98us/step
Epoch 1/5
6000/6000 [==================

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  6.9min finished


Epoch 1/5
8000/8000 [==============================] - 6s 766us/step - loss: 0.5032 - accuracy: 0.7454
Epoch 2/5
8000/8000 [==============================] - 6s 742us/step - loss: 0.2460 - accuracy: 0.9029
Epoch 3/5
8000/8000 [==============================] - 6s 752us/step - loss: 0.1020 - accuracy: 0.9724
Epoch 4/5
8000/8000 [==============================] - 6s 743us/step - loss: 0.0268 - accuracy: 0.9977
Epoch 5/5
8000/8000 [==============================] - 6s 746us/step - loss: 0.0073 - accuracy: 0.9999


In [28]:
print(grid_result.best_score_)
print(grid_result.best_params_)
test_accuracy = grid.score(X_test_pad, y_test)
print(test_accuracy)

0.8743750005960464
{'vocab_size': 54487, 'num_filters': 128, 'maxlen': 1830, 'kernel_size': 5, 'embedding_dim': 100}
2000/2000 [==============================] - 0s 180us/step
0.8730000257492065


In [30]:
embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=max_length, 
                           ))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_44"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_44 (Embedding)     (None, 1830, 100)         5448700   
_________________________________________________________________
conv1d_44 (Conv1D)           (None, 1826, 128)         64128     
_________________________________________________________________
global_max_pooling1d_44 (Glo (None, 128)               0         
_________________________________________________________________
dense_87 (Dense)             (None, 10)                1290      
_________________________________________________________________
dense_88 (Dense)             (None, 1)                 11        
Total params: 5,514,129
Trainable params: 5,514,129
Non-trainable params: 0
_________________________________________________________________


In [39]:
model.fit(X_train_pad,y_train,epochs=5,batch_size=64,validation_split=0.2)

Train on 6400 samples, validate on 1600 samples
Epoch 1/5
6400/6400 [==============================] - 4s 664us/step - loss: 4.3749e-04 - accuracy: 1.0000 - val_loss: 6.3243e-04 - val_accuracy: 1.0000
Epoch 2/5
6400/6400 [==============================] - 4s 661us/step - loss: 3.9479e-04 - accuracy: 1.0000 - val_loss: 6.0918e-04 - val_accuracy: 1.0000
Epoch 3/5
6400/6400 [==============================] - 4s 662us/step - loss: 3.5792e-04 - accuracy: 1.0000 - val_loss: 5.9273e-04 - val_accuracy: 1.0000
Epoch 4/5
6400/6400 [==============================] - 4s 656us/step - loss: 3.2523e-04 - accuracy: 1.0000 - val_loss: 5.8948e-04 - val_accuracy: 1.0000
Epoch 5/5
6400/6400 [==============================] - 4s 659us/step - loss: 2.9767e-04 - accuracy: 1.0000 - val_loss: 5.5899e-04 - val_accuracy: 1.0000


In [50]:
from sklearn.metrics import confusion_matrix,accuracy_score
y_pred = model.predict(X_test_pad)
y_pred = y_pred.round()
y_test = y_test.astype('float32')
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

[[838 141]
 [102 919]]
0.8785
